In [1]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [2]:
# load input features and make layer
version='v10_240710'
output_dir = r'J:\lakemapping\postprocess\v10_240710'
after_mask_gdb=os.path.join(output_dir,'3_polygon_after_rivermask.gdb')
stastics_gdb=os.path.join(output_dir,'0_summary_stastics.gdb')
Agdb=os.path.join(output_dir,'a_polygon_after_mask_Continent.gdb')
Bgdb=os.path.join(output_dir,'b_polygon_afm_mergeWith_BigLake_Continent.gdb')
Cgdb=os.path.join(output_dir,'c_polygon_afm_mergeWith_BL_BigGLAKES_Continent.gdb')
if not os.path.exists(Agdb):
    arcpy.CreateFileGDB_management(output_dir,'a_polygon_after_mask_Continent.gdb')
if not os.path.exists(Bgdb):
    arcpy.CreateFileGDB_management(output_dir,'b_polygon_afm_mergeWith_BigLake_Continent.gdb')
if not os.path.exists(Cgdb):
    arcpy.CreateFileGDB_management(output_dir,'c_polygon_afm_mergeWith_BL_BigGLAKES_Continent.gdb')
if not os.path.exists(stastics_gdb):
    arcpy.CreateFileGDB_management(output_dir,'0_summary_stastics.gdb')

In [37]:
a_iwBL_merge_with_BL_raw=os.path.join(Agdb,'a_iwBL_merge_with_BL_raw')
a_iwBL_merge_with_BL=os.path.join(Agdb,'a_iwBL_merge_with_BL')
a_iwBL_merge_with_BL_raw_lyr='a_iwBL_merge_with_BL_raw'
a_iwBL_merge_with_BL_lyr='a_iwBL_merge_with_BL'
b_iwBG_merge_with_BG_raw=os.path.join(Bgdb,'b_iwBG_merge_with_BG_raw')
b_iwBG_merge_with_BG=os.path.join(Bgdb,'b_iwBG_merge_with_BG')
b_iwBG_merge_with_BG_raw_lyr='b_iwBG_merge_with_BG_raw'
b_iwBG_merge_with_BG_lyr='b_iwBG_merge_with_BG'


In [15]:
seven_continents=['Asia','Africa','Europe','Oceania_Anta','North_America','South_America']


In [4]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

In [5]:
region_list=['eu','af','oc','island','greenland','sa_1','sa_2']
for i in range(1,10):
    region_list.append(f'as_{i}')
    
for i in range(1,7):
    region_list.append(f'na_{i}')

In [6]:
big_lakes=os.path.join(Agdb,f'big_lakes')
big_lakes_lyr=f'big_lakes'
GLAKES_gte1=os.path.join(Bgdb,'GLAKES_gte1')
GLAKES_lt1=os.path.join(Bgdb,'GLAKES_lt1')
GLAKES_gte1_lyr='GLAKES_gte1'
GLAKES_lt1_lyr='GLAKES_gte1'

In [7]:
continent=r'D:\lakemapping\0_auxiliary_data\World_Continents\World_Continents.shp'
continent_lyr='continent'
arcpy.MakeFeatureLayer_management(continent,continent_lyr)

<Result 'continent'>

## 1， 对GLAKES进行分类，合并big_lakes

In [32]:
# GLAKES=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES\GLAKES.gdb\GLAKES'
# arcpy.MakeFeatureLayer_management(GLAKES,'GLAKES')
# selectByAttribute('GLAKES','NEW_SELECTION','"Area_bound" >= 1')
# arcpy.conversion.ExportFeatures('GLAKES',GLAKES_gte1)
# selectByAttribute('GLAKES','SWITCH_SELECTION','')
# arcpy.conversion.ExportFeatures('GLAKES',GLAKES_lt1)

<Result 'J:\\lakemapping\\postprocess\\v10_240710\\b_polygon_afm_mergeWith_BigLake_Continent.gdb\\GLAKES_lt1'>

In [8]:
# big_lake_list=[]
# for region in region_list:
#     polygon_big_lake=os.path.join(after_mask_gdb,'big_lakes_'+region)
#     arcpy.MakeFeatureLayer_management(polygon_big_lake,'big_lakes_'+region)
#     big_lake_list.append('big_lakes_'+region)
# arcpy.Merge_management(big_lake_list,big_lakes)

## 2，将mask后结果与big_lakes合并，得到的数据存在Bgdb

In [9]:
arcpy.MakeFeatureLayer_management(big_lakes,big_lakes_lyr)

<Result 'big_lakes'>

In [26]:
## 添加 Continent 及 iwBL 属性
start=time.time()
input_table_afm_list=[]
for region in region_list:
    lakes_afm=os.path.join(after_mask_gdb,'lakes_'+region+'_afterMask')
    lakes_afm_lyr='lakes_'+region+'_afm'
    arcpy.MakeFeatureLayer_management(lakes_afm,lakes_afm_lyr)
    arcpy.AddField_management(lakes_afm_lyr, 'Continent', "Text")
    arcpy.AddField_management(lakes_afm_lyr, 'iwBL', "Short")
    selectByLocation(lakes_afm_lyr,'INTERSECT',big_lakes_lyr)
    calculateField(lakes_afm_lyr,'iwBL',1)
    for i in range(0,6):
        continent=seven_continents[i]
        selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
        selectByAttribute(lakes_afm_lyr,'NEW_SELECTION',"Continent IS NULL")
        selectByLocation(lakes_afm_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
        calculateField(lakes_afm_lyr,'Continent',f"'{continent}'")
    selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','')
    out_table=os.path.join(stastics_gdb,f'{region}_afm_by_Continent_stastics')
    input_table_afm_list.append(out_table)
    arcpy.analysis.Statistics(lakes_afm_lyr, out_table, [['area',"sum"]], ['Continent']) 
    selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','iwBL IS NULL')
    calculateField(lakes_afm_lyr,'iwBL',0)
end=time.time()
print(f'time:{end-start}s')

time:746.0724091529846s


In [28]:
Output_Excel_File=r'J:\lakemapping\postprocess\v10_240710\sum_afm_classify_by_Continents_stastics.xls'
arcpy.conversion.TableToExcel(input_table_afm_list, Output_Excel_File)

<Result 'J:\\lakemapping\\postprocess\\v10_240710\\sum_afm_classify_by_Continents_stastics.xls'>

In [9]:
j=16

In [25]:
# region='island'
# lakes_afm=os.path.join(after_mask_gdb,'lakes_'+region+'_afterMask')
# lakes_afm_lyr='lakes_'+region+'_afm'
# arcpy.MakeFeatureLayer_management(lakes_afm,lakes_afm_lyr)
#     arcpy.AddField_management(lakes_afm_lyr, 'Continent', "Text")
#     arcpy.AddField_management(lakes_afm_lyr, 'iwBL', "Short")
#     selectByLocation(lakes_afm_lyr,'INTERSECT',big_lakes_lyr)
#     calculateField(lakes_afm_lyr,'iwBL',1)
# for i in range(0,6):
#     continent=seven_continents[i]
#     selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
#     selectByAttribute(lakes_afm_lyr,'NEW_SELECTION',"Continent IS NULL")
#     selectByLocation(lakes_afm_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
#     calculateField(lakes_afm_lyr,'Continent',f"'{continent}'")
# selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','')
# out_table=os.path.join(stastics_gdb,f'{region}_afm_by_Continent_stastics')
# input_table_afm_list.append(out_table)
# arcpy.analysis.Statistics(lakes_afm_lyr, out_table, [['area',"sum"]], ['Continent']) 
# j=j+1
# arcpy.Delete_management(lakes_afm_lyr)

<Result 'J:\\lakemapping\\postprocess\\v10_240710\\0_summary_stastics.gdb\\island_afm_by_Continent_stastics'>

In [10]:
afm_list=[]
for region in region_list:
    lakes_afm_lyr='lakes_'+region+'_afm'
    afm_list.append(lakes_afm_lyr)
    selectByAttribute(lakes_afm_lyr,'NEW_SELECTION',"iwBL = 1")

In [12]:
#把iwBL与big lake文件合并，并融合
merge_list=afm_list.copy()
merge_list.append(big_lakes_lyr)
print(merge_list)
start=time.time()
arcpy.Merge_management(merge_list,a_iwBL_merge_with_BL_raw)
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL_raw, a_iwBL_merge_with_BL_raw_lyr)#global_lakes_afm_merged_with_GBL
end=time.time()
print(f'time:{end-start}s')

['lakes_eu_afm', 'lakes_af_afm', 'lakes_oc_afm', 'lakes_island_afm', 'lakes_greenland_afm', 'lakes_sa_1_afm', 'lakes_sa_2_afm', 'lakes_as_1_afm', 'lakes_as_2_afm', 'lakes_as_3_afm', 'lakes_as_4_afm', 'lakes_as_5_afm', 'lakes_as_6_afm', 'lakes_as_7_afm', 'lakes_as_8_afm', 'lakes_as_9_afm', 'lakes_na_1_afm', 'lakes_na_2_afm', 'lakes_na_3_afm', 'lakes_na_4_afm', 'lakes_na_5_afm', 'lakes_na_6_afm', 'big_lakes']
time:41.49833416938782s


In [13]:
start=time.time()
print('start Dissolve')
arcpy.Dissolve_management(a_iwBL_merge_with_BL_raw_lyr, a_iwBL_merge_with_BL,"", "", "SINGLE_PART")#dissolve copylake
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr,"area","Double")
arcpy.CalculateField_management(a_iwBL_merge_with_BL_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
end=time.time()
print(f'time:{end-start}s')

start Dissolve
time:1702.3034601211548s


In [30]:
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)

<Result 'a_iwBL_merge_with_BL'>

In [16]:
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'Continent', "Text")
for i in range(0,6):
    continent=seven_continents[i]
    selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',"Continent IS NULL")
    selectByLocation(a_iwBL_merge_with_BL_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
    calculateField(a_iwBL_merge_with_BL_lyr,'Continent',f"'{continent}'")

In [38]:
b_lyr_list=[]
for i in range(0,6):
    continent=seven_continents[i]
    continent_merge_list=[a_iwBL_merge_with_BL_lyr]
    for j in continent_region_list[i]:
        selectByAttribute(afm_list[j],'NEW_SELECTION',f"Continent = '{continent}' AND iwBL = 0")
        continent_merge_list.append(afm_list[j])
    print(continent_merge_list)
    selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    b_lake_afm_merge_with_BL=os.path.join(Bgdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
    b_lyr_list.append(b_lake_afm_merge_with_BL_lyr)
#     arcpy.Merge_management(continent_merge_list,b_lake_afm_merge_with_BL)
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    arcpy.AddField_management(b_lake_afm_merge_with_BL_lyr, 'iwBG', "Short")
    selectByLocation(b_lake_afm_merge_with_BL_lyr,'INTERSECT',GLAKES_gte1_lyr)
    calculateField(b_lake_afm_merge_with_BL_lyr,'iwBG',1)
    selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','iwBG IS NULL')
    calculateField(b_lake_afm_merge_with_BL_lyr,'iwBG',0)

## 3，将b结果与big_GLAKES合并，得到的数据存在Cgdb

In [39]:
#3.1 把iwBG与big GLAKES文件合并，并融合
merge_list_BG=b_lyr_list.copy()
merge_list_BG.append(GLAKES_gte1_lyr)
print(merge_list_BG)

start=time.time()
# for i in range(0,6):
#     b_lyr=b_lyr_list[i]
#     selectByAttribute(b_lyr,'NEW_SELECTION',"iwBG = 1")
    
arcpy.Merge_management(merge_list_BG,b_iwBG_merge_with_BG_raw)
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_raw, b_iwBG_merge_with_BG_raw_lyr)#global_lakes_afm_merged_with_GBL
end=time.time()
print(f'time:{end-start}s')

['b1_Asia_lake_afm_merge_with_BL', 'b2_Africa_lake_afm_merge_with_BL', 'b3_Europe_lake_afm_merge_with_BL', 'b4_Oceania_Anta_lake_afm_merge_with_BL', 'b5_North_America_lake_afm_merge_with_BL', 'b6_South_America_lake_afm_merge_with_BL', 'GLAKES_gte1']
time:111.72716164588928s


In [40]:
start=time.time()
print('start Dissolve')
arcpy.Dissolve_management(b_iwBG_merge_with_BG_raw_lyr, b_iwBG_merge_with_BG,"", "", "SINGLE_PART")#dissolve copylake
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG, b_iwBG_merge_with_BG_lyr)
arcpy.AddField_management(b_iwBG_merge_with_BG_lyr,"area","Double")
arcpy.CalculateField_management(b_iwBG_merge_with_BG_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
end=time.time()
print(f'time:{end-start}s')

start Dissolve
time:4815.872866868973s


In [42]:
arcpy.AddField_management(b_iwBG_merge_with_BG_lyr, 'Continent', "Text")
for i in range(1,6):
    continent=seven_continents[i]
    selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    selectByAttribute(b_iwBG_merge_with_BG_lyr,'NEW_SELECTION',"Continent IS NULL")
    selectByLocation(b_iwBG_merge_with_BG_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
    calculateField(b_iwBG_merge_with_BG_lyr,'Continent',f"'{continent}'")

In [44]:
## 3.2 将niwGB与 3.1的结果合并生成c
for i in range(0,6):
    b_lyr=b_lyr_list[i]
    continent=seven_continents[i]
    arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
    arcpy.analysis.Statistics(b_lyr, out_table, [['area',"sum"]], ['iwBG']) 
    selectByAttribute(b_lyr,'NEW_SELECTION',"iwBG = 0")
    selectByAttribute(b_iwBG_merge_with_BG_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    c_lake_afm_mw_BL_BG=os.path.join(Bgdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
    c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
    arcpy.Merge_management([b_lyr,b_iwBG_merge_with_BG_lyr],c_lake_afm_mw_BL_BG)
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
    

In [52]:
input_table_mwBL_list=[]
input_table_mwBL_BG_list=[]
for i in range(0,6):
    b_lyr=b_lyr_list[i]
    continent=seven_continents[i]
    out_table_1=os.path.join(stastics_gdb,f'b{i+1}_{continent}_afm_mwBL_by_Continent_stastics')
    input_table_mwBL_list.append(out_table_1)
    
    c_lake_afm_mw_BL_BG=os.path.join(Bgdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
    c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
#     arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
    out_table=os.path.join(stastics_gdb,f'c{i+1}_{continent}_afm_mwBL_BG_by_Continent_stastics')
    input_table_mwBL_BG_list.append(out_table)
#     arcpy.analysis.Statistics(c_lake_afm_mw_BL_BG_lyr, out_table, [['area',"sum"]]) 
    out_table_2=os.path.join(stastics_gdb,f'c{i+1}_{continent}_afm_mwBL_BG_by_Continent_stastics')
    input_table_mwBL_BG_list.append(out_table_2)
    
Output_Excel_File=r'J:\lakemapping\postprocess\v10_240710\sum_afm_mwBL_classify_by_Continents_stastics.xls'
Output_Excel_File_2=r'J:\lakemapping\postprocess\v10_240710\sum_afm_mwBL_BG_classify_by_Continents_stastics.xls'
print(input_table_mwBL_BG_list)
arcpy.conversion.TableToExcel(input_table_mwBL_list, Output_Excel_File)
arcpy.conversion.TableToExcel(input_table_mwBL_BG_list, Output_Excel_File_2)

b1_Asia_lake_afm_merge_with_BL
b2_Africa_lake_afm_merge_with_BL
b3_Europe_lake_afm_merge_with_BL
b4_Oceania_Anta_lake_afm_merge_with_BL
b5_North_America_lake_afm_merge_with_BL
b6_South_America_lake_afm_merge_with_BL
